As electronic vehicles (EVs) become more popular, there is an increasing need for access to charging stations, also known as ports. To that end, many modern apartment buildings have begun retrofitting their parking garages to include shared charging stations. A charging station is shared if it is accessible by anyone in the building.


But with increasing demand comes competition for these ports — nothing is more frustrating than coming home to find no charging stations available! In this project, you will use a dataset to help apartment building managers better understand their tenants’ EV charging habits.

The data has been loaded into a PostgreSQL database with a table named `charging_sessions` with the following columns:

## charging_sessions

| Column | Definition | Data type |
|-|-|-|
|`garage_id`| Identifier for the garage/building|`VARCHAR`|
|`user_id` | Identifier for the individual user|`VARCHAR`|
|`user_type`|Indicating whether the station is `Shared` or `Private`| `VARCHAR` |
|`start_plugin`|The date and time the session started |`DATETIME`|
|`start_plugin_hour`|The hour (in military time) that the session started | `NUMERIC`|
|`end_plugout`|The date and time the session ended | `DATETIME` |
|`end_plugout_hour`|The hour (in military time) that the session ended | `NUMERIC`|
|`duration_hours`| The length of the session, in hours|`NUMERIC`|
|`el_kwh`| Amount of electricity used (in Kilowatt hours)|`NUMERIC`|
|`month_plugin`| The month that the session started |`VARCHAR`|
|`weekdays_plugin`| The day of the week that the session started|`VARCHAR`|

Let’s get started!

#### Sources
- **Data**: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via [Kaggle](https://www.kaggle.com/datasets/anshtanwar/residential-ev-chargingfrom-apartment-buildings),
- **Image**: Julian Herzog, [CC BY 4.0](https://creativecommons.org/licenses/by/4.0), via Wikimedia Commons

## 1. Unique Users per Garage

This query calculates the number of unique users who used shared charging stations in each garage. It groups the data by `garage_id` and counts distinct `user_id` values to ensure only unique users are included. The results are sorted in descending order of the number of unique users per garage.


In [40]:
-- unique_users_per_garage
SELECT garage_id, COUNT(DISTINCT user_id) AS num_unique_users
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY garage_id
ORDER BY num_unique_users DESC

,garage_id,num_unique_users
0,Bl2,18
1,AsO2,17
2,UT9,16
3,AdO3,3
4,MS1,2
5,SR2,2
6,AdA1,1
7,Ris,1


## 2. Top 10 Most Popular Charging Start Times

This query identifies the top 10 most popular charging start times for sessions using shared charging stations. It groups sessions by the day of the week (weekdays_plugin) and the hour the charging began (start_plugin_hour), then counts the number of sessions in each group. The results are sorted by the number of sessions (num_charging_sessions) in descending order, showing the most popular times for charging first.

In [41]:
-- most_popular_shared_start_times
SELECT weekdays_plugin, start_plugin_hour, COUNT(*) AS num_charging_sessions
FROM charging_sessions
WHERE user_type = 'Shared'
GROUP BY weekdays_plugin, start_plugin_hour
ORDER BY num_charging_sessions DESC
LIMIT 10;


,weekdays_plugin,start_plugin_hour,num_charging_sessions
0,Sunday,17,30
1,Friday,15,28
2,Thursday,19,26
3,Thursday,16,26
4,Wednesday,19,25
5,Sunday,18,25
6,Sunday,15,25
7,Monday,15,24
8,Friday,16,24
9,Tuesday,16,23


## 3.Identify Users with Long Average Charging Durations

This query finds users whose average charging duration exceeds 10 hours while using shared charging stations. The output includes the user ID and their average charging duration, sorted in descending order. This analysis helps identify users with prolonged charging behavior, which could impact shared station availability and efficiency.

In [42]:
-- long_duration_shared_users
SELECT user_id, 
	AVG(duration_hours) as avg_charging_duration
FROM charging_sessions 
WHERE user_type = 'Shared' 
GROUP BY user_id
HAVING AVG(duration_hours) > 10
ORDER BY AVG(duration_hours) DESC;

,user_id,avg_charging_duration
0,Share-9,16.845833
1,Share-17,12.894556
2,Share-25,12.214475
3,Share-18,12.088807
4,Share-8,11.550431
5,AdO3-1,10.369387
